# Lecture7 重回帰分析（血圧と年齢の関係）
- "Linear regression using the Framingham Heart Study and StatsModels package"より
- https://github.com/RM-Santiago/framingham

収縮期血圧（sBP）と年齢との臨床的な関連性を評価する疫学研究を実施する予定である。
これまでのデータでは、sBPは年齢とともに増加する傾向にあります。つまり、年をとればとるほど、平均sBPは高くなるのです。

この実験は、フラミンガムコホート（最初で最大の疫学的コホートの1つ - https://en.wikipedia.org/wiki/Framingham_Heart_Study ）のデータを活用し、StatsModelsパッケージを使ってPythonでいくつかのモデルを実行する

臨床的な問い:
### 潜在的な臨床的交絡因子を考慮した場合、年齢と平均sBPの間に線形相関があるのか？
選択した交絡因子：性別、教育、喫煙、血圧降下剤摂取、総コレステロール値

* **帰無仮説 (H0)**: sBPと年齢との間に関連はない (considered the chosen confounders);
* **対立仮説 (H1)**: sBPと年齢には関連性がある (considered the chosen confounders).

We are going to extract a publically available version of the Framingham heart study from; https://www.kaggle.com/amanajmera1/framingham-heart-study-dataset/version/1

On this website, you can also consult the coding for the different variables. For example, regarding gender (variable male; male=1 means male and male=2 means female)

### 使用するパッケージのインポート

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# EDA:データ探索

### データ読み込み(githubサイトのcsvファイルを　DataFrameに読み込む）

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2024/main/notebooks/data/framingham.csv')

### データフレーム表示

In [ ]:
df.head()

最初の５行だけが表示された

In [ ]:
df.shape

データは4240行で、16変数(コラム)であることがわかる

In [ ]:
df.info()

In [ ]:
df.describe()

** カテゴリー変数： Education

** 離散２値変数; male, CurrentSmoker, BPMeds, prevalentStroke, prevalentHyp, diabetes and TenYearCHD.


### 単回帰の実施

まず、結果/従属変数である**sysBP**と主な従属変数である**age**の間の関連を**StatsModel パッケージ**を活用して検定する

回帰OLS実施と、引数について

**model1 = smf.ols(formula='sysBP ~ age', data=df).fit()**

In [ ]:
model1 = smf.ols(formula='sysBP ~ age', data=df).fit()

In [ ]:
print(model1.summary())

summaryを読み解くと、

* 左部分には、問題設定が、
   * 目的変数は sysBP
   * 最小二乗法OLSを
   * 患者数は4240人.

* 右半分には、モデル診断に関する情報が乗っている、
   * R-squared = 0.155,  このモデルによって、全体の16% の変動が説明可能である。The R-squared は 0–1の範囲で, データがどれだけ回帰曲線に近いかを示す。 言い換えれば、結果（応答データ）の変動のうち、何％がモデルによって説明されるかを示すものである;
   * F-statistic value がこのモデルのp値を計算している。すなわち、Prob (F-statistic)が<0.05.

下半分について

出力を一列ずつ読んでいきます。ここでは、我々の結果（sysbp）と我々の独立変数である年齢（1列目）の間の関連を評価しています。これは、1.0128（または係数、2列目）の正の関連を持ち、標準誤差は0. 036（3列目）、t比率（t分布の場合）27.911（4列目）で、p値<0.001（5列目）、ベータ係数の95%信頼区間は、0.94 2（下限、6列目）と1.084（上限、7列目）に対応することがわかります。もっと簡単に言うと

### 年齢とsBPの間には関連がないという帰無仮説を棄却することができる。年齢と収縮期血圧の間に正の（β係数>1.0）関連があり、これは統計的に有意である（p<0.01、95%CIには係数=0.0は含まれない）。

### 重回帰の実施

今度は交絡因子を追加します。言い換えると、1つ以上の独立変数を追加します（関数の"~"の右側）。

In [ ]:
model2 = smf.ols(formula='sysBP ~ age + Sex + education + cigsPerDay + BPMeds + totChol', data=df).fit()

In [ ]:
print(model2.summary())

###  "education" をカテゴリー変数に変換します

education は4つのカテゴリーをもっています

In [ ]:
df.education.value_counts()

Sexもそうか

In [ ]:
df.Sex.value_counts()

educationがカテゴリ変数であることをmodel式で明記します。

In [ ]:
model3 = smf.ols(formula='sysBP ~ age + C(Sex)+ C(education) + cigsPerDay + BPMeds + totChol', data=df).fit()

In [ ]:
print(model3.summary())

Same model in another style

In [ ]:
model3.summary()

完璧です、それでは、この結果から、最初の臨床的な質問に対して、以下のように答えることができます。

## 収縮期血圧と年齢の間に関連性はありますか？

### はい、年齢と収縮期血圧の間には正の相関があり（ベータ係数[95%CI]：0.84[0.76;0.91]）、関連しうる交絡因子（モデル中の共変量）の影響を考慮しても統計的に有意（p<0.05、信頼区間に0は含まない）であることがわかりました。

### References

 https://www.w3schools.com/python/python_ml_multiple_regression.asp
 Run a regression More info here: https://www.statsmodels.org/stable/glm.html